In [ ]:
import logging
 
logger = logging.getLogger()
logger.setLevel(logging.INFO)  # Log等级总开关
log_file_abs = "flask.log"
 
stream_handler = logging.StreamHandler()  # 日志控制台输出
 
handler = logging.FileHandler(log_file_abs, mode='w', encoding='UTF-8')  # 日志文件输出
handler.setLevel(logging.DEBUG)
 
# 控制台输出格式
stream_format = logging.Formatter("Time: %(asctime)s -- INFO: %(message)s")
 
# 文件输出格式
logging_format = logging.Formatter("Time: %(asctime)s -- INFO: %(message)s")
 
handler.setFormatter(logging_format)  # 为改处理器handler 选择一个格式化器
stream_handler.setFormatter(stream_format)
 
logger.addHandler(handler)  # 为记录器添加 处理方式Handler
logger.addHandler(stream_handler)
 
logger.info("------logger.info-----")

In [ ]:
import time
print(time.localtime())

In [ ]:
print("{:.6f}".format(1))

In [ ]:
from model.transunet import TransUNet
from utils import OhemCrossEntropy2dTensor
import torch
vit = TransUNet(img_dim=512,
                          in_channels=3,
                          out_channels=128,
                          head_num=4,
                          mlp_dim=512,
                          block_num=8,
                          patch_dim=16,
                          class_num=2)

# vit(torch.rand(1, 3, 512, 512)).shape
optimizer = torch.optim.SGD(vit.parameters(), lr=0.01,momentum=0.9,weight_decay=0.0005)
print(optimizer)
scheduler=torch.optim.lr_scheduler.ExponentialLR(optimizer,gamma=0.5)
print(scheduler.state_dict())
a = torch.nn.CrossEntropyLoss(reduction="None")
print(a)

In [ ]:
def function1():
    print("function1")

def function2():
    print("function2")

def function3():
    print("function3")

def call_fun_by_str(fun_str):
    eval(fun_str)()

call_fun_by_str("function2")

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

x = torch.rand(2,64,128,128)

# y = F.interpolate(x,size=(128,128), scale_factor=None, mode='bilinear', align_corners=None, recompute_scale_factor=None)
# print(y.shape)
    
trans_conv = nn.ConvTranspose2d(in_channels=64, out_channels=16,kernel_size=2, stride=2)
y = trans_conv(x)
print(y.shape)

# pixel = nn.PixelShuffle(4)
# y = pixel(x)
# print(y.shape)

In [2]:
import torch
import segmentation_models_pytorch as smp
model1 = smp.DeepLabV3Plus(
        encoder_name="resnet34",        # choose encoder, e.g. mobilenet_v2 or efficientnet-b7
        encoder_weights="imagenet",     # use `imagenet` pre-trained weights for encoder initialization
        in_channels=3,                  # model input channels (1 for gray-scale images, 3 for RGB, etc.)
        classes=2,                      # model output channels (number of classes in your dataset)
    )

model2 = smp.Unet(
    encoder_name="resnet34",        # choose encoder, e.g. mobilenet_v2 or efficientnet-b7
    encoder_weights="imagenet",     # use `imagenet` pre-trained weights for encoder initialization
    in_channels=3,                  # model input channels (1 for gray-scale images, 3 for RGB, etc.)
    classes=2,                      # model output channels (number of classes in your dataset)
)
print(model2)

Unet(
  (encoder): ResNetEncoder(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (1): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track

In [ ]:
class ChannelAttention(nn.Module):
    def __init__(self, in_planes, ratio=8):
        super(ChannelAttention, self).__init__()
        self.avg_pool = nn.AdaptiveAvgPool2d(1)
        self.max_pool = nn.AdaptiveMaxPool2d(1)
 
        # 利用1x1卷积代替全连接
        self.fc1   = nn.Conv2d(in_planes, in_planes // ratio, 1, bias=False)
        self.relu1 = nn.ReLU()
        self.fc2   = nn.Conv2d(in_planes // ratio, in_planes, 1, bias=False)
 
        self.sigmoid = nn.Sigmoid()
 
    def forward(self, x):
        avg_out = self.fc2(self.relu1(self.fc1(self.avg_pool(x))))
        max_out = self.fc2(self.relu1(self.fc1(self.max_pool(x))))
        out = avg_out + max_out
        return self.sigmoid(out)
 
class SpatialAttention(nn.Module):
    def __init__(self, kernel_size=7):
        super(SpatialAttention, self).__init__()
 
        assert kernel_size in (3, 7), 'kernel size must be 3 or 7'
        padding = 3 if kernel_size == 7 else 1
        self.conv1 = nn.Conv2d(2, 1, kernel_size, padding=padding, bias=False)
        self.sigmoid = nn.Sigmoid()
 
    def forward(self, x):
        avg_out = torch.mean(x, dim=1, keepdim=True)
        max_out, _ = torch.max(x, dim=1, keepdim=True)
        x = torch.cat([avg_out, max_out], dim=1)
        x = self.conv1(x)
        return self.sigmoid(x)
 
class cbam_block(nn.Module):
    def __init__(self, channel, ratio=8, kernel_size=7):
        super(cbam_block, self).__init__()
        self.channelattention = ChannelAttention(channel, ratio=ratio)
        self.spatialattention = SpatialAttention(kernel_size=kernel_size)
 
    def forward(self, x):
        x = x * self.channelattention(x)
        x = x * self.spatialattention(x)
        return x

In [2]:
import torch
a = torch.nn.Conv2d(3, 256, kernel_size=7, stride=4,
                              padding=(7 // 2, 7 // 2))
x = torch.rand(1,3,512,512)
y = a(x)
print(y.shape)

torch.Size([1, 256, 128, 128])


In [3]:
import torch
from thop import profile
from model import transunet,swinunet,deeplabv3p_smp,unet_smp,m_segformer_T,pspnet_smp
model = pspnet_smp()
x = torch.randn(2, 3, 512, 512)
flops , params = profile(model,inputs=(x,))
print(flops)
print(params)

[INFO] Register count_convNd() for <class 'torch.nn.modules.conv.Conv2d'>.
[INFO] Register zero_ops() for <class 'torch.nn.modules.container.Sequential'>.
[INFO] Register count_normalization() for <class 'torch.nn.modules.batchnorm.BatchNorm2d'>.
[INFO] Register zero_ops() for <class 'torch.nn.modules.activation.ReLU'>.
[INFO] Register count_upsample() for <class 'torch.nn.modules.upsampling.Upsample'>.
23826792448.0
9811970.0
